# Task 4

Estimate the expected lifetime, after surgery, of a woman who survives the first 12 months following surgery, but whose breast cancer has also reappeared within the first 12 months, either locally or distant.

_Hint:_ Use rejection sampling. Simulate a number of women, and discard all simulations that do not meet the requirements. Do this until you have reached 1000 acceptable simulations.

**Solution:** Just to make it more clear the 5 states are the following:
1. Women with breast tumor removed, and are healty.
2. Cancer reappears locally (cancer reappears close to the removed tumor).
3. Cancer reappears distant from where it was operated (called distant metastatis).
4. Cancer both reappears locally and distant at same time.
5. Death.

So, we are asked to estimate the expected lifetime, after surgery, of women who survive first 12 months but have entered state 2 or state 3 within the first 12 months. 

**Note:** We made the assumption that it was either state 2 or 3 that had to have appeared in the first 12 months, so we don't consider state 4 where the women has both a local and distant reappearance.

This is implemented in the code below:

In [73]:
# Our markov chain simulation, with given n: number of women and P: probability matrix (transition matrix)
def am_12_months(n, P):
    lifetime = []
    accepted_sims = 0
    while accepted_sims < n:
        state = []
        t = 0 # Time variable controlling the 12 months horizon, we start in first month
        acceptable = False # If the simulation is acceptable or not
        
        # Initial state
        current_state = 0 # state 1 --> index 0
        state.append(current_state) # first state which is always state 1
        
        while current_state != 4:  # Absorbing state (death state)
            t += 1
            # Making a jump based on p=P[current state], returning the state index (state nr. -1)
            next_state = np.random.choice([0, 1, 2, 3, 4], p=P[current_state])
                
            state.append(next_state) # Keeping records of which state the women is in
            current_state = next_state # Change of state to next state

            if (current_state == 1 or current_state == 2) and t <= 12:
                acceptable = True
            
            if current_state == 4 and acceptable and t > 12: # Death state, but has entered either state 2 or 3 in
                lifetime.append(state)                       # first 12 months and has survived more than 12 months           
                accepted_sims += 1

    return lifetime

# Probability matrix P with probabilities:
P = np.array([
    [0.9915, 0.005, 0.0025, 0.0, 0.001],  # state 1 to all the other
    [0.0, 0.986, 0.005, 0.004, 0.005],   # state 2: local recurrence
    [0.0, 0.0, 0.992, 0.003, 0.005],   # state 3:  distant metastatis
    [0.0, 0.0, 0.0, 0.991, 0.009],   # state 4: both local recurrence and distant metastatis
    [0.0, 0.0, 0.0, 0.0, 1.0]    # state 5: death state, no return hence 1.0 probabilites in P_[4][4]
])

nr_women = 1_000
## Simulating ##
lifetime = am_12_months(nr_women, P)


In [85]:
lifetimes_month = []
for i in range(len(lifetime)):
    lifetime_list = lifetime[i]
    lifetimes_month.append(len(lifetime_list))

lifetimes = np.array(lifetimes_month) # Calculating the lifetime of each women, in months
expected_lifetime = np.mean(lifetimes)

print(f"The expected lifetime is {expected_lifetime:.2f} in months or {expected_lifetime/12:.2f} in years.")

The expected lifetime is 172.67 in months or 14.39 in years.
